## Filter out low signal audio files by clustering CQT representations

Clustering to remove low signal sample

In [49]:
from time import time
import numpy as np
np.random.seed(10)
import numpy as np
import keras.backend as K
from keras.engine.topology import Layer, InputSpec
from keras.layers import Dense, Input
from keras.models import Model
from keras.optimizers import SGD
from keras import callbacks
from keras.initializers import VarianceScaling
from sklearn.cluster import KMeans
import metrics

In [50]:
import scipy.io 
from scipy.io import wavfile
import librosa

In [51]:
import os
drive = "D:\\"
cwd_top = os.path.join(drive,"bird-data")
cwd_source = os.path.join(cwd_top,"test-set-resampled-audio")
cwd_target_cqt = os.path.join(cwd_top, "cqt-full-sample")
os.chdir(cwd_source)
os.getcwd()

'D:\\bird-data\\test-set-resampled-audio'

In [52]:
sample_list = os.listdir(cwd_source)
len(sample_list)

677

In [54]:
count = 0
primary_input = []

for samples in sample_list:
    count = count + 1
    os.chdir(cwd_source)
    [fs, s] = wavfile.read(samples) 
    s = np.asarray(s[0:228800])
    
    constant_q = np.abs(librosa.cqt(s, sr=22000, fmin=librosa.note_to_hz('C1'), n_bins=70, bins_per_octave=12))
    constant_q_shape_1 = constant_q.shape[1]
    constant_q_shape_0 = constant_q.shape[0]
    primary_input.append(constant_q)
    os.chdir(cwd_target_cqt)
    samples_without_extension = samples[:-4]
    # np.savetxt(samples_without_extension+'_CQT_'+str(count)+'.csv', constant_q, delimiter=',')

primary_input = np.array(primary_input).reshape(-1, constant_q_shape_0, constant_q_shape_1)
primary_input.shape  

(677, 70, 447)

In [55]:
 x = primary_input.reshape((primary_input.shape[0], -1))
print("segment cqt shape", x.shape)

segment cqt shape (677, 31290)


In [56]:
os.getcwd()

'D:\\bird-data\\cqt-full-sample'

In [57]:
np.savetxt('unrolled-unfiltered-cqt'+'.csv', x, delimiter=',')  

In [58]:
sample_list = np.asarray(sample_list)
sample_list.shape

(677,)

In [59]:

np.savetxt('cqt_file_list'+'.csv', sample_list, delimiter=',', fmt="%s")  

In [60]:
import pandas as pd
# x=pd.read_csv('cleaned_cqt_unrolled.csv', sep=',',header=None)
# x.shape

In [61]:
n_clusters = 8

## Base line K-Means clustering accuracy

In [62]:
kmeans = KMeans(n_clusters=n_clusters, n_init=20, n_jobs=4)
y_pred_kmeans = kmeans.fit_predict(x)

C:\Anaconda\envs\recommender\lib\site-packages\sklearn\cluster\_kmeans.py:968: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 0.25.
  " removed in 0.25.", FutureWarning)


In [63]:
titles=pd.read_csv('cqt_file_list.csv', sep=',',header=None)
titles.shape


(677, 1)

In [64]:
y_pred_kmeans.shape


(677,)

In [65]:
import collections 
  
# it returns a dictionary data structure whose  
# keys are array elements and values are their  
# corresponding frequencies {1: 4, 2: 4, 3: 2,  
# 5: 2, 4: 1} 
def CountFrequency(arr): 
    return collections.Counter(arr)   
  
# Driver function 
if __name__ == "__main__": 
  
    freq = CountFrequency(y_pred_kmeans) 
    print(freq)


Counter({7: 597, 1: 39, 4: 36, 3: 1, 0: 1, 5: 1, 6: 1, 2: 1})


In [66]:
y_pred_kmeans = pd.DataFrame(y_pred_kmeans)
# y_pred_kmeans['Row_Num'] = y_pred_kmeans.reset_index().index

# y_pred_kmeans.columns = ["Cluster","Row_Num"]
# y_pred_kmeans = y_pred_kmeans[["Row_Num","Cluster"]]
# print(y_pred_kmeans.head(2))
# y_pred_kmeans.shape

In [67]:
titles = pd.DataFrame(titles)
y_pred_kmeans.shape

(677, 1)

In [68]:
cqt_clusters = pd.concat([titles, y_pred_kmeans], axis=1)

In [69]:
cqt_clusters.shape

(677, 2)

In [70]:
cqt_clusters.head(2)
cqt_clusters = np.asarray(cqt_clusters)
cqt_clusters.shape

(677, 2)

In [72]:
np.savetxt('cleaned_cqt_clusters.csv', cqt_clusters, delimiter=',', fmt='%s')  